In [ ]:
import React, { useState, useEffect, useCallback, useRef } from 'react';
import {
  Button,
  Text,
  Spinner,
  makeStyles,
  tokens,
} from "@fluentui/react-components";
import * as FluentIcons from "@fluentui/react-icons";
import SubgraphResults from './SubgraphResults';

const useStyles = makeStyles({
  // ... (keep existing styles)

  chatPage: {
    display: "flex",
    flexDirection: "column",
    height: "100vh",
    padding: "24px",
    boxSizing: "border-box",
  },
  chatMessages: {
    flexGrow: 1,
    overflowY: "auto",
    display: "flex",
    flexDirection: "column",
    gap: "16px",
    marginBottom: "16px",
  },
  subgraphMessage: {
    width: "100%",
    padding: "12px",
    backgroundColor: tokens.colorNeutralBackground2,
    borderRadius: tokens.borderRadiusMedium,
  },
  // ... (keep other existing styles)
});

const DynamicProgressBar = ({ currentStep }) => {
  // ... (keep existing implementation)
};

const EmailGenerator = () => {
  const styles = useStyles();
  // ... (keep existing state variables)

  const handleUpdate = useCallback((data) => {
    console.log("handleUpdate received data:", data);
    if (data.type === 'step') {
      setCurrentStep(data.content);
    } else if (data.type === 'final') {
      finalResponseRef.current = data.content;
      setChatMessages(prevMessages => [
        ...prevMessages,
        { role: "assistant", content: wrapInHtml(data.content) }
      ]);
    } else if (data.type === 'subgraph_results') {
      setChatMessages(prevMessages => [
        ...prevMessages,
        { role: "subgraph", content: data.content }
      ]);
    }
  }, []);

  const handleAction = useCallback(async (userInput) => {
    setIsProcessing(true);
    setError(null);
    setStatusMessage(null);
    setCurrentStep("");
    finalResponseRef.current = null;
    
    setChatMessages(prevMessages => [...prevMessages, { role: "user", content: userInput }]);
    
    try {
      // ... (keep existing API call logic)
    } catch (e) {
      console.error(`Error in handleAction: ${e.message}`);
      setError('Failed to process your request. Please try again.');
    } finally {
      setIsProcessing(false);
      setCurrentStep("");
    }
  }, [userConfig, getEmailContent, handleUpdate]);

  // ... (keep other existing functions)

  if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => {
            if (message.role === "subgraph") {
              return (
                <div key={index} className={styles.subgraphMessage}>
                  <SubgraphResults results={message.content} />
                </div>
              );
            }
            return (
              <div
                key={index}
                className={`${styles.chatMessage} ${
                  message.role === "user" ? styles.userMessage : styles.assistantMessage
                }`}
              >
                {message.role === "user" ? (
                  message.content
                ) : (
                  <div dangerouslySetInnerHTML={{ __html: message.content }} />
                )}
              </div>
            );
          })}
        </div>
        {isProcessing && (
          <DynamicProgressBar currentStep={currentStep} />
        )}
        <form onSubmit={handleChatSubmit} className={styles.chatInput}>
          <input
            type="text"
            value={chatInput}
            onChange={(e) => setChatInput(e.target.value)}
            placeholder="Type your message..."
            className={styles.chatInputField}
            disabled={isProcessing}
          />
          <Button type="submit" disabled={isProcessing || !chatInput.trim()}>
            Send
          </Button>
        </form>
      </div>
    );
  }

  // ... (keep the rest of the component, including the return statement for the initial view)
};

export default EmailGenerator;